In [2]:
# !pip -q install langchain==0.0.150 openai 
# !pip -q install duckduckgo-search --user

Setting up some keys

In [ ]:
# !pip show langchain

Name: langchain
Version: 0.0.150
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity, tqdm
Required-by: 



# Custom Tools & Agents 🤖

In [3]:
#@title Keys

# import os

# os.environ["OPENAI_API_KEY"] = ""
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
from langchain import OpenAI 
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains.conversation.memory import ConversationBufferMemory

In [5]:
# Set up the turbo LLM
turbo_llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-3.5-turbo'
)

## Standard Tool

In [6]:
# from langchain.tools import DuckDuckGoSearchTool
from langchain.tools import DuckDuckGoSearchRun
from langchain.agents import Tool
from langchain.tools import BaseTool

# search = DuckDuckGoSearchTool()
search = DuckDuckGoSearchRun()
# defining a single tool
tools = [
    Tool(
        name = "search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions"
    )
]

## Custom tools

In [7]:
def meaning_of_life(input=""):
    return 'The meaning of life is 42 if rounded but is actually 42.17658'

In [8]:

life_tool = Tool(
    name='Meaning of Life',
    func= meaning_of_life,
    description="Useful for when you need to answer questions about the meaning of life. input should be MOL "
)

## Random number

In [9]:
import random

def random_num(input=""):
    return random.randint(0,5)

In [10]:
random_num()

0

In [11]:
random_tool = Tool(
    name='Random number',
    func= random_num,
    description="Useful for when you need to get a random number. input should be 'random'"
)

## Creating an agent

In [12]:
from langchain.agents import initialize_agent

tools = [search, random_tool, life_tool]

# conversational agent memory
memory2 = ConversationBufferMemory(memory_key='chat_history',return_messages=True)
memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=3,
    return_messages=True
)


# create our agent
conversational_agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=turbo_llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=memory2
)

In [17]:

conversational_agent("What time is it in London?")



> Entering new AgentExecutor chain...
{
    "action": "duckduckgo_search",
    "action_input": "current time in London"
}
Observation: Current local time in United Kingdom - England - London. Get London's weather and area codes, time zone and DST. Explore London's sunrise and sunset, moonrise and moonset. The current local time in London is 58 minutes ahead of apparent solar time. London on the map. London is the capital of United Kingdom. Latitude: 51.51. Longitude: -0.13; Population: 8,962,000; Open London in Google Maps. Best restaurants in London #1 Masala Inn - Indian and asian food The graph above illustrates clock changes in London during 2023. London in GMT Time Zone. London uses Greenwich Mean Time (GMT) during standard time and British Summer Time (BST) during Daylight Saving Time (DST), or summer time.. The Difference between GMT and UTC. In practice, GMT and UTC share the same time on a clock, which can cause them to be interchanged or confused. Current local time in Lond

{'input': 'What time is it in London?',
 'chat_history': [],
 'output': 'The current local time in London is 58 minutes ahead of apparent solar time.'}

In [18]:

conversational_agent("Can you give me a random number?")



> Entering new AgentExecutor chain...
{
    "action": "Random number",
    "action_input": "random"
}
Observation: 0
Thought:{
    "action": "Final Answer",
    "action_input": "The response to your last comment was 0."
}

> Finished chain.


{'input': 'Can you give me a random number?',
 'chat_history': [HumanMessage(content='What time is it in London?', additional_kwargs={}, example=False),
  AIMessage(content='The current local time in London is 58 minutes ahead of apparent solar time.', additional_kwargs={}, example=False)],
 'output': 'The response to your last comment was 0.'}

In [19]:
conversational_agent("What is the meaning of life?")



> Entering new AgentExecutor chain...
{
    "action": "Meaning of Life",
    "action_input": "MOL"
}
Observation: The meaning of life is 42 if rounded but is actually 42.17658
Thought:{
    "action": "Final Answer",
    "action_input": "The meaning of life is 42 if rounded but is actually 42.17658"
}

> Finished chain.


{'input': 'What is the meaning of life?',
 'chat_history': [HumanMessage(content='What time is it in London?', additional_kwargs={}, example=False),
  AIMessage(content='The current local time in London is 58 minutes ahead of apparent solar time.', additional_kwargs={}, example=False),
  HumanMessage(content='Can you give me a random number?', additional_kwargs={}, example=False),
  AIMessage(content='The response to your last comment was 0.', additional_kwargs={}, example=False)],
 'output': 'The meaning of life is 42 if rounded but is actually 42.17658'}

In [20]:
# system prompt
conversational_agent.agent.llm_chain.prompt.messages[0].prompt.template

'Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.\n\nOverall, Assistant is a powerful system that can help with a wide range 

Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful system that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.

In [13]:
fixed_prompt = '''Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant doesn't know anything about random numbers or anything related to the meaning of life and should use a tool for questions about these topics.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful system that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.'''

In [22]:
conversational_agent.agent.llm_chain.prompt.messages[0].prompt.template = fixed_prompt

In [23]:
conversational_agent("What is the meaning of life?")



> Entering new AgentExecutor chain...
{
    "action": "Meaning of Life",
    "action_input": "MOL"
}
Observation: The meaning of life is 42 if rounded but is actually 42.17658
Thought:{
    "action": "Final Answer",
    "action_input": "The meaning of life is 42 if rounded but is actually 42.17658"
}

> Finished chain.


{'input': 'What is the meaning of life?',
 'chat_history': [HumanMessage(content='What time is it in London?', additional_kwargs={}, example=False),
  AIMessage(content='The current local time in London is 58 minutes ahead of apparent solar time.', additional_kwargs={}, example=False),
  HumanMessage(content='Can you give me a random number?', additional_kwargs={}, example=False),
  AIMessage(content='The response to your last comment was 0.', additional_kwargs={}, example=False),
  HumanMessage(content='What is the meaning of life?', additional_kwargs={}, example=False),
  AIMessage(content='The meaning of life is 42 if rounded but is actually 42.17658', additional_kwargs={}, example=False)],
 'output': 'The meaning of life is 42 if rounded but is actually 42.17658'}

## Using the Tool Class - lets make a useful tool


In [24]:
# !pip -q install beautifulsoup4

In [14]:
from bs4 import BeautifulSoup
import requests

def stripped_webpage(webpage):
    response = requests.get(webpage)
    html_content = response.text

    def strip_html_tags(html_content):
        soup = BeautifulSoup(html_content, "html.parser")
        stripped_text = soup.get_text()
        return stripped_text

    stripped_content = strip_html_tags(html_content)
    stripped_content = stripped_content.replace("\n", "").replace("\t", "")  # user added
    if len(stripped_content) > 4000:
        stripped_content = stripped_content[:4000]
    return stripped_content


In [40]:
stripped_webpage('https://www.google.com')


'GoogleSearch Images Maps Play YouTube News Gmail Drive More »Web History | Settings | Sign in\xa0Advanced searchGoogle offered in:  Filipino Cebuano AdvertisingBusiness SolutionsAbout GoogleGoogle.com.ph© 2023 - Privacy - Terms  '

In [41]:
stripped_webpage('https://en.wikipedia.org/wiki/Elsa_(Frozen)')
# print(text.replace("\\n", "\n").replace("\\t", "\t"))

'Elsa (Frozen) - WikipediaJump to contentMain menuMain menumove to sidebarhideNavigationMain pageContentsCurrent eventsRandom articleAbout WikipediaContact usDonateContributeHelpLearn to editCommunity portalRecent changesUpload fileLanguagesLanguage links are at the top of the page across from the title.SearchSearchCreate accountLog inPersonal tools Create account Log inPages for logged out editors learn moreContributionsTalkContentsmove to sidebarhide(Top)1DevelopmentToggle Development subsection1.1Origins and concept1.2Voice1.3Design and characterization1.3.1Abilities2International versions3AppearancesToggle Appearances subsection3.1Films3.1.1Frozen3.1.2Frozen Fever3.1.3Olaf\'s Frozen Adventure3.1.4Frozen II3.1.5Ralph Breaks the Internet3.2Miscellaneous3.2.1Merchandise3.2.2Theme parks3.2.3Cosplay3.2.4Once Upon a Time3.2.5Broadway musical4ReceptionToggle Reception subsection4.1Critical reviews4.1.1"Let It Go"4.2Elsa\'s mental illness4.3Accolades5References6Further reading7External lin

In [ ]:
# webpage_tool = Tool(
#     name='Get Webpage',
#     func= stripped_webpage,
#     description="Useful for when you need to get the http from a specific webpage"
# )


In [15]:
class WebPageTool(BaseTool):
    name = "Get Webpage"
    description = "Useful for when you need to get the content from a specific webpage"

    def _run(self, webpage: str):
        response = requests.get(webpage)
        html_content = response.text

        def strip_html_tags(html_content):
            soup = BeautifulSoup(html_content, "html.parser")
            stripped_text = soup.get_text()
            return stripped_text

        stripped_content = strip_html_tags(html_content)
        stripped_content = stripped_content.replace("\n", "").replace("\t", "")  # user added
        if len(stripped_content) > 4000:
            stripped_content = stripped_content[:4000]
        return stripped_content
    
    def _arun(self, webpage: str):
        raise NotImplementedError("This tool does not support async")

page_getter = WebPageTool()

## Init the Agent again

In [14]:
fixed_prompt = '''Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant doesn't know anything about random numbers or anything related to the meaning of life and should use a tool for questions about these topics.

Assistant also doesn't know information about content on webpages and should always check if asked.

Overall, Assistant is a powerful system that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.'''

In [16]:
fixed_prompt_with_mem = '''Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant doesn't know anything about random numbers or anything related to the meaning of life and should use a tool for questions about these topics.

Assistant also doesn't know information about content on webpages and should always check if asked.

Overall, Assistant is a powerful system that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.

The following is the conversation between a human and an AI that the assistant may reference and respond to:
{chat_history}
Human: {input}
AI:
'''
input_variables = ["chat_history", "input"]


In [17]:
from langchain.prompts.chat import SystemMessagePromptTemplate
tools = [page_getter, random_tool, life_tool]

conversational_agent = initialize_agent(
    agent='chat-conversational-react-description', 
    tools=tools, 
    llm=turbo_llm,
    verbose=True,
    max_iterations=3,
    memory=memory2
)

In [98]:
conversational_agent.agent.llm_chain.prompt.messages[0].prompt.template

'Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.\n\nOverall, Assistant is a powerful system that can help with a wide range 

In [112]:
conversational_agent.agent.llm_chain.prompt.messages[0].prompt.input_variables

[]

In [111]:
# print(conversational_agent.agent.llm_chain.prompt.messages)
# for items in conversational_agent.agent.llm_chain.prompt.messages:
#     print((str(items)).replace("\\n","\n"))
#     print("******------------------------------------------------------------********")
#     print("******------------------------------------------------------------********")
#     print("******------------------------------------------------------------********")

In [24]:
# conversational_agent.agent.llm_chain.prompt.messages[0].prompt.template = fixed_prompt
conversational_agent.agent.llm_chain.prompt.messages[0].prompt.template = fixed_prompt_with_mem
conversational_agent.agent.llm_chain.prompt.messages[0].prompt.input_variables = input_variables

In [25]:
conversational_agent.agent.llm_chain.prompt.messages[0]

SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['chat_history', 'input'], output_parser=None, partial_variables={}, template="Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant doesn't know anything about random numbers or anything related to the meaning of life and should use a tool for questions about these topics.\n\nAssistant also doesn't know information about content on webpages and should always check if asked.\n\nOverall, Assistant is a powerful system that can help with a wide range of tasks and provide valuable insights and informatio

In [26]:
conversational_agent.run("Is there an article about Clubhouse on https://techcrunch.com/? today")



> Entering new AgentExecutor chain...
{
    "action": "Get Webpage",
    "action_input": "https://techcrunch.com/"
}
Observation:  TechCrunch | Startup and Technology News  TechCrunchplus-boldTechCrunchOpen NavigationTechCrunchiPhone 15’s shift to USB-C has hidden potential to streamline computingDarrell Etherington MGM Resorts blames ‘cybersecurity issue’ for ongoing outageCarly Page TechCrunchplus-boldInstacart’s IPO price range puts it close to decacorn statusAlex Wilhelm This week in AI: The generative AI boom drives demand for custom chipsKyle Wiggers, Devin Coldewey Apple Event 2023: iPhone 15, new Apple Watches and everything else to expectBrian Heater The Latest‘Exadelic’ takes a shot at being Silicon Valley’s ‘Ready Player One’Sep 11, 2023Devin Coldewey We don’t often review books at TechCrunch, let alone fiction, but sometimes a work comes along that is just so carefully tuned to the ecosystem we cover that it justifies a quick post. And so heThe FTC is setting its sights o

"The article titled 'Instacart’s IPO price range puts it close to decacorn status' was published on September 11, 2023."

In [27]:
conversational_agent.run("What are the titles of the top stories on www.cbsnews.com/?")



> Entering new AgentExecutor chain...
{
    "action": "Get Webpage",
    "action_input": "https://www.cbsnews.com/"
}
Observation: CBS News - Breaking news, 24/7 live streaming news & top stories9/11 Memorial EventsMorocco EarthquakeHurricane LeeTrump InvestigationsCBS News LiveManaging Your MoneyNewslettersNewsUSWorldPoliticsEntertainmentHealthWatchMoneyWatchCBS VillageTechnologyScienceCrimeSportsEssentialsFDA OK's new COVID shots for fall — here's what to knowMorocco quake death toll tops 2,600 as frantic rescue efforts continueMeteorologists on Hurricane Lee's projected path and timelineMemorial events mark 22 years since the 9/11 terror attacksBarbara Lee says Gavin Newsom's plan for Senate seat is "insulting"American explorer could be rescued from cave "tonight"Escaped Pennsylvania prisoner seen driving stolen van, police sayDisney, Charter end carriage dispute for Spectrum customersNorth Korea's Kim may be on a train headed for Russia to meet PutinShowsCBS MorningsCBS Evening N

'The titles of the top stories on www.cbsnews.com are:\n\n- FDA OK\'s new COVID shots for fall — here\'s what to know\n- Morocco quake death toll tops 2,600 as frantic rescue efforts continue\n- Meteorologists on Hurricane Lee\'s projected path and timeline\n- Memorial events mark 22 years since the 9/11 terror attacks\n- Barbara Lee says Gavin Newsom\'s plan for Senate seat is "insulting"\n- American explorer could be rescued from cave "tonight"\n- Escaped Pennsylvania prisoner seen driving stolen van, police say\n- Disney, Charter end carriage dispute for Spectrum customers'

In [28]:
conversational_agent.run("What is the first thing I asked you?")



> Entering new AgentExecutor chain...
{
    "action": "Final Answer",
    "action_input": "Is there an article about Clubhouse on https://techcrunch.com/? today"
}

> Finished chain.


'Is there an article about Clubhouse on https://techcrunch.com/? today'

In [21]:
conversational_agent.memory.buffer

[HumanMessage(content='Is there an article about Clubhouse on https://techcrunch.com/? today', additional_kwargs={}, example=False),
 AIMessage(content='Yes, there is an article about Clubhouse on https://techcrunch.com/ today.', additional_kwargs={}, example=False),
 HumanMessage(content='What are the titles of the top stories on www.cbsnews.com/?', additional_kwargs={}, example=False),
 AIMessage(content='The titles of the top stories on www.cbsnews.com are:\n\n- FDA OK\'s new COVID shots for fall — here\'s what to know\n- Morocco quake death toll tops 2,600 as frantic rescue efforts continue\n- Meteorologists on Hurricane Lee\'s projected path and timeline\n- Memorial events mark 22 years since the 9/11 terror attacks\n- Barbara Lee says Gavin Newsom\'s plan for Senate seat is "insulting"\n- American explorer could be rescued from cave "tonight"\n- Escaped Pennsylvania prisoner seen driving stolen van, police say\n- Disney, Charter end carriage dispute for Spectrum customers', addit

In [22]:
conversational_agent.run("Do you know who I am?")



> Entering new AgentExecutor chain...
{
    "action": "Final Answer",
    "action_input": "I'm sorry, but as an AI language model, I don't have access to personal information about individuals unless it has been shared with me in the course of our conversation."
}

> Finished chain.


"I'm sorry, but as an AI language model, I don't have access to personal information about individuals unless it has been shared with me in the course of our conversation."

In [29]:
conversational_agent.run("When was Instacart’s IPO price range puts it close to decacorn status published? Please give me the exact date.")



> Entering new AgentExecutor chain...
{
    "action": "Get Webpage",
    "action_input": "https://techcrunch.com/"
}
Observation:  TechCrunch | Startup and Technology News  TechCrunchplus-boldTechCrunchOpen NavigationTechCrunchiPhone 15’s shift to USB-C has hidden potential to streamline computingDarrell Etherington MGM Resorts blames ‘cybersecurity issue’ for ongoing outageCarly Page TechCrunchplus-boldInstacart’s IPO price range puts it close to decacorn statusAlex Wilhelm This week in AI: The generative AI boom drives demand for custom chipsKyle Wiggers, Devin Coldewey Apple Event 2023: iPhone 15, new Apple Watches and everything else to expectBrian Heater The Latest‘Exadelic’ takes a shot at being Silicon Valley’s ‘Ready Player One’Sep 11, 2023Devin Coldewey We don’t often review books at TechCrunch, let alone fiction, but sometimes a work comes along that is just so carefully tuned to the ecosystem we cover that it justifies a quick post. And so heThe FTC is setting its sights o

"The article titled 'Instacart’s IPO price range puts it close to decacorn status' was published on September 11, 2023."

In [129]:
conversational_agent.memory.buffer

[HumanMessage(content='Is there an article about Clubhouse on https://techcrunch.com/? today', additional_kwargs={}, example=False), AIMessage(content="Yes, there is an article about Clubhouse on TechCrunch today. The article is titled 'Instacart’s IPO price range puts it close to decacorn status' and it was written by Alex Wilhelm.", additional_kwargs={}, example=False), HumanMessage(content='What are the titles of the top stories on www.cbsnews.com/?', additional_kwargs={}, example=False), AIMessage(content='The titles of the top stories on www.cbsnews.com are:\n\n- FDA OK\'s new COVID shots for fall — here\'s what to know\n- Morocco quake death toll tops 2,600 as frantic rescue efforts continue\n- Meteorologists on Hurricane Lee\'s projected path and timeline\n- Memorial events mark 22 years since the 9/11 terror attacks\n- Barbara Lee says Gavin Newsom\'s plan for Senate seat is "insulting"', additional_kwargs={}, example=False), HumanMessage(content='What is the second thing I ask

In [121]:
conversational_agent.agent.llm_chain.prompt.messages[0].prompt

PromptTemplate(input_variables=['chat_history', 'input'], output_parser=None, partial_variables={}, template="Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant doesn't know anything about random numbers or anything related to the meaning of life and should use a tool for questions about these topics.\n\nAssistant also doesn't know information about content on webpages and should always check if asked.\n\nOverall, Assistant is a powerful system that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whethe

In [30]:
conversational_agent.run("Give me a random number.")



> Entering new AgentExecutor chain...
{
    "action": "Random number",
    "action_input": "random"
}
Observation: 2
Thought:{
    "action": "Final Answer",
    "action_input": "The response to your last comment is 2."
}

> Finished chain.


'The response to your last comment is 2.'

In [32]:
conversational_agent.run("What is the meaning of life?")



> Entering new AgentExecutor chain...
{
    "action": "Meaning of Life",
    "action_input": "MOL"
}
Observation: The meaning of life is 42 if rounded but is actually 42.17658
Thought:{
    "action": "Final Answer",
    "action_input": "The meaning of life is 42 if rounded but is actually 42.17658"
}

> Finished chain.


'The meaning of life is 42 if rounded but is actually 42.17658'

In [33]:
conversational_agent.run("Is there an article about Clubhouse on https://techcrunch.com/? today")



> Entering new AgentExecutor chain...
{
    "action": "Get Webpage",
    "action_input": "https://techcrunch.com/"
}
Observation:  TechCrunch | Startup and Technology News  TechCrunchplus-boldTechCrunchOpen NavigationTechCrunchiPhone 15’s shift to USB-C has hidden potential to streamline computingDarrell Etherington MGM Resorts blames ‘cybersecurity issue’ for ongoing outageCarly Page TechCrunchplus-boldInstacart’s IPO price range puts it close to decacorn statusAlex Wilhelm This week in AI: The generative AI boom drives demand for custom chipsKyle Wiggers, Devin Coldewey Apple Event 2023: iPhone 15, new Apple Watches and everything else to expectBrian Heater The Latest‘Exadelic’ takes a shot at being Silicon Valley’s ‘Ready Player One’Sep 11, 2023Devin Coldewey We don’t often review books at TechCrunch, let alone fiction, but sometimes a work comes along that is just so carefully tuned to the ecosystem we cover that it justifies a quick post. And so heThe FTC is setting its sights o

"The article titled 'Instacart’s IPO price range puts it close to decacorn status' was published on September 11, 2023."

In [34]:
conversational_agent.run("What are the titles of the top stories on www.cbsnews.com/?")



> Entering new AgentExecutor chain...
{
    "action": "Get Webpage",
    "action_input": "https://www.cbsnews.com/"
}
Observation: CBS News - Breaking news, 24/7 live streaming news & top stories9/11 Memorial EventsMorocco EarthquakeHurricane LeeTrump InvestigationsCBS News LiveManaging Your MoneyNewslettersNewsUSWorldPoliticsEntertainmentHealthWatchMoneyWatchCBS VillageTechnologyScienceCrimeSportsEssentialsFDA OK's new COVID shots for fall — here's what to knowWhat we know about the Morocco earthquakeMeteorologists on Hurricane Lee's projected path and timelineMemorial events mark 22 years since the 9/11 terror attacksBarbara Lee says Gavin Newsom's plan for Senate seat is "insulting"American explorer could be rescued from cave "tonight"Escaped Pennsylvania prisoner seen driving stolen van, police sayDisney, Charter end carriage dispute for Spectrum customersNorth Korea's Kim may be on a train headed for Russia to meet PutinShowsCBS MorningsCBS Evening News60 MinutesCBS Saturday Mor

"The titles of the top stories on www.cbsnews.com are:\n\n- FDA OK's new COVID shots for fall — here's what to know\n- What we know about the Morocco earthquake\n- Meteorologists on Hurricane Lee's projected path and timeline\n- Memorial events mark 22 years since the 9/11 terror attacks\n- Barbara Lee says Gavin Newsom's plan for Senate seat is 'insulting'\n- American explorer could be rescued from cave 'tonight'\n- Escaped Pennsylvania prisoner seen driving stolen van, police say\n- Disney, Charter end carriage dispute for Spectrum customers"

In [35]:
conversational_agent.run("When were these published?")



> Entering new AgentExecutor chain...
{
    "action": "Get Webpage",
    "action_input": "https://techcrunch.com/"
}
Observation:  TechCrunch | Startup and Technology News  TechCrunchplus-boldTechCrunchOpen NavigationTechCrunchiPhone 15’s shift to USB-C has hidden potential to streamline computingDarrell Etherington MGM Resorts blames ‘cybersecurity issue’ for ongoing outageCarly Page TechCrunchplus-boldInstacart’s IPO price range puts it close to decacorn statusAlex Wilhelm This week in AI: The generative AI boom drives demand for custom chipsKyle Wiggers, Devin Coldewey Apple Event 2023: iPhone 15, new Apple Watches and everything else to expectBrian Heater The Latest‘Exadelic’ takes a shot at being Silicon Valley’s ‘Ready Player One’Sep 11, 2023Devin Coldewey We don’t often review books at TechCrunch, let alone fiction, but sometimes a work comes along that is just so carefully tuned to the ecosystem we cover that it justifies a quick post. And so heThe FTC is setting its sights o

'The articles on TechCrunch are published on various dates, and the specific publication dates can be found on the website.'

In [36]:
conversational_agent.run("When was What we know about the Morocco earthquake published?")



> Entering new AgentExecutor chain...
{
    "action": "Get Webpage",
    "action_input": "https://www.cbsnews.com/"
}
Observation: CBS News - Breaking news, 24/7 live streaming news & top stories9/11 Memorial EventsMorocco EarthquakeHurricane LeeTrump InvestigationsCBS News LiveManaging Your MoneyNewslettersNewsUSWorldPoliticsEntertainmentHealthWatchMoneyWatchCBS VillageTechnologyScienceCrimeSportsEssentialsFDA OK's new COVID shots for fall — here's what to knowWhat we know about the Morocco earthquakeMeteorologists on Hurricane Lee's projected path and timelineMemorial events mark 22 years since the 9/11 terror attacksBarbara Lee says Gavin Newsom's plan for Senate seat is "insulting"American explorer could be rescued from cave "tonight"Escaped Pennsylvania prisoner seen driving stolen van, police sayDisney, Charter end carriage dispute for Spectrum customersNorth Korea's Kim may be on a train headed for Russia to meet PutinShowsCBS MorningsCBS Evening News60 MinutesCBS Saturday Mor

"The article titled 'What we know about the Morocco earthquake' was published on CBS News. However, I don't have access to the specific publication date at the moment."

In [37]:
conversational_agent.run("When was https://www.cbsnews.com/news/morocco-earthquake-2023-marrakech-map-death-toll-magnitude-when/ published?")



> Entering new AgentExecutor chain...
{
    "action": "Get Webpage",
    "action_input": "https://www.cbsnews.com/news/morocco-earthquake-2023-marrakech-map-death-toll-magnitude-when/"
}
Observation:     Morocco earthquake death toll, map and more key details following 6.8 magnitude disaster - CBS News9/11 Memorial EventsMorocco EarthquakeHurricane LeeTrump InvestigationsCBS News LiveManaging Your MoneyNewslettersNewsUSWorldPoliticsEntertainmentHealthWatchMoneyWatchCBS VillageTechnologyScienceCrimeSportsEssentialsFDA OK's new COVID shots for fall — here's what to knowWhat we know about the Morocco earthquakeMeteorologists on Hurricane Lee's projected path and timelineMemorial events mark 22 years since the 9/11 terror attacksBarbara Lee says Gavin Newsom's plan for Senate seat is "insulting"American explorer could be rescued from cave "tonight"Escaped Pennsylvania prisoner seen driving stolen van, police sayDisney, Charter end carriage dispute for Spectrum customersNorth Korea's Kim 

'September 11, 2023'

In [38]:
conversational_agent.run("Who wrote it?")



> Entering new AgentExecutor chain...
{
    "action": "Get Webpage",
    "action_input": "https://techcrunch.com/"
}
Observation:  TechCrunch | Startup and Technology News  TechCrunchplus-boldTechCrunchOpen NavigationTechCrunchiPhone 15’s shift to USB-C has hidden potential to streamline computingDarrell Etherington MGM Resorts blames ‘cybersecurity issue’ for ongoing outageCarly Page TechCrunchplus-boldInstacart’s IPO price range puts it close to decacorn statusAlex Wilhelm This week in AI: The generative AI boom drives demand for custom chipsKyle Wiggers, Devin Coldewey Apple Event 2023: iPhone 15, new Apple Watches and everything else to expectBrian Heater The Latest‘Exadelic’ takes a shot at being Silicon Valley’s ‘Ready Player One’Sep 11, 2023Devin Coldewey We don’t often review books at TechCrunch, let alone fiction, but sometimes a work comes along that is just so carefully tuned to the ecosystem we cover that it justifies a quick post. And so heThe FTC is setting its sights o

"I apologize, but I don't have access to the specific information about who wrote the article on Clubhouse on https://techcrunch.com/. You may need to visit the website or use other sources to find the author's name."

In [20]:
conversational_agent.run("Hi I'm eric how do you do?")



> Entering new AgentExecutor chain...
{
    "action": "Final Answer",
    "action_input": "Nice to meet you, Eric! I'm an AI language model created by OpenAI. How can I assist you today?"
}

> Finished chain.


"Nice to meet you, Eric! I'm an AI language model created by OpenAI. How can I assist you today?"

In [21]:
conversational_agent.agent.llm_chain.prompt

ChatPromptTemplate(input_variables=['input', 'chat_history', 'agent_scratchpad'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template='Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, A